In [33]:
# Jozef Porubcin
# Semester Project
# Code adapted from Practical 2: Skeleton credit to Yifan Qin, Zheng Ning, and Adam Czajka

In [34]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms.functional import resize
from torchvision.transforms import CenterCrop
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torchsummary import summary
from tqdm import tqdm
import numpy as np
import pandas as pd

import csv
import random

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
class CNN(nn.Module):
    def __init__(self, numChannels, numClasses):
        super(CNN, self).__init__()
        self.classes = numClasses
        
        conv_out1 = 96
        conv_out2 = 256
        conv_out3 = 384
        conv_out4 = 384
        conv_out5 = 256
        self.conv1 = torch.nn.Conv2d(numChannels, conv_out1, (11, 11), (4, 4))
        self.conv2 = torch.nn.Conv2d(conv_out1, conv_out2, (5, 5), (1, 1))
        self.conv3 = torch.nn.Conv2d(conv_out2, conv_out3, (3, 3), (1, 1))
        self.conv4 = torch.nn.Conv2d(conv_out3, conv_out4, (3, 3), (1, 1))
        self.conv5 = torch.nn.Conv2d(conv_out4, conv_out5, (3, 3), (1, 1))

        self.relu = torch.nn.ReLU()

        self.maxpool = torch.nn.MaxPool2d((2, 2), (2, 2))

        self.batchnorm1 = torch.nn.BatchNorm2d(conv_out1)
        self.batchnorm2 = torch.nn.BatchNorm2d(conv_out2)

        fc1_out1 = 1024
        self.fc1 = torch.nn.Linear(conv_out5*15, fc1_out1)
        self.fc2 = torch.nn.Linear(fc1_out1, numClasses)

    def evaluate(self, model, dataloader, classes, device):

        # We need to switch the model into the evaluation mode
        model.eval()
        
        correct_pred = {classname: 0 for classname in classes}
        total_pred = {classname: 0 for classname in classes}

        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)

            images = images.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
            predictions = predictions.detach().cpu().numpy()

            for label, prediction in zip(labels, predictions):
                
                # If you want to see real and predicted labels for all samples:
                # print("Real class: " + classes[label] + ", predicted = " + classes[prediction])
                
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

        acc = sum(correct_pred.values()) / sum(total_pred.values())

        return acc


    def forward(self, x):
        x = resize(x, size=[256])

        # First convolutional block
        # self.conv1 -> self.relu -> self.maxpool -> self.batchnorm1
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.batchnorm1(x)

        # Second convolutional block
        # self.conv2 -> self.relu -> self.maxpool -> self.batchnorm2
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.batchnorm2(x)

        # Third convolutional block
        # self.conv3 -> self.relu
        x = self.conv3(x)
        x = self.relu(x)

        # Fourth convolutional block
        # self.conv4 -> self.relu
        x = self.conv4(x)
        x = self.relu(x)

        # Fifth convolutional block
        # self.conv5 -> self.relu -> self.maxpool
        x = self.conv5(x)
        x = self.relu(x)
        x = self.maxpool(x)


        # After the last pooling operation, and before the first 
        # fully-connected layer, we need to "flatten" our tensors
        x = torch.flatten(x, 1)


        # self.fc1 -> self.relu -> self.fc2
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

In [37]:
# Check which layers are "trainable"
def check(model):
    print("Layers to be fine-tuned:")
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print(name)


In [38]:
class FilteredFruits(Dataset):
    def __init__(self, img_dir, label_dir, transform):
        self.img_dir = img_dir
        self.img_labels = pd.read_csv(label_dir, header=0)
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = image.to(torch.float) / 256.
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)

        return (image, label)

In [39]:
def generate_labels(image_dir, label_dir):
    row_list = [['file', 'label']]

    min_images = float('inf')
    for class_dir in os.scandir(image_dir):
        if class_dir.is_dir():
            min_images = min(len(os.listdir(class_dir)), min_images)

    l = 0
    for class_dir in os.scandir(image_dir):
        if class_dir.is_dir():
            img_files = os.listdir(class_dir)
            random.shuffle(img_files)
            for x in range(min_images):
                row_list.append([(class_dir.name + '/' + img_files[x]), l])

            l += 1
    
    with open(label_dir, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(row_list)

In [44]:
if __name__ == '__main__':
    # Transform images:
    # a) to tensor: convert the PIL image or numpy.ndarray to tensor
    # b) Z-normalize a tensor image (using its mean and standard deviation)
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    
    # Path to the fruits dataset:
    image_dir = '/content/drive/MyDrive/Semester_Project/208x256/'

    # Path to the labels of the sampled CIFAR-100 subset
    label_dir = '/content/drive/MyDrive/Semester_Project/208x256/fruits_labels.csv'

    generate_labels(image_dir, label_dir)

    transfer_learning = 'all_layers'

    epochs = 50
    batch_size = 20

    # Load sampled fruits dataset
    data = FilteredFruits(img_dir=image_dir, label_dir=label_dir, transform=transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)))
    train_len = int(len(data) * 0.8)
    val_len = int(len(data) * 0.1)
    test_len = int(len(data) - train_len - val_len)
    train_data, val_data, test_data = random_split(data, [train_len, val_len, test_len])
    classes = ['apple', 'banana', 'black berry', 'blueberry', 'cherry', 'grape', 'mango', 'orange',  'pear', 'pineapple', 'pineberry', 'pomegranate', 'raspberry', 'strawberry', 'watermelon']

    # Prepare data loaders for train, validation and test data splits 
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)

    #######################################################
    # Specify the operation mode:
    # 'train' = training with your train and validation data splits
    # 'eval'  = evaluation of the trained model with your test data split
    mode = 'eval'

    # Path where you will save the best model after fine-tuning
    my_best_model = "/content/drive/MyDrive/Semester_Project/best_model.pth"

    #
    #######################################################


    # Set the device (GPU or CPU, depending on availability)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Currently using device: ", device)

    # Initialize the model
    model_t = CNN(numChannels = 3, numClasses = len(classes))
    model_t.to(device)
    
    # Training happens here!
    if mode == "train":

        print("\n\nFine-tuning (transfer learning) starts!\n\n")

        model_t.train()
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.Adam(model_t.parameters(), lr=0.001)
        optimizer = torch.optim.SGD(model_t.parameters(), lr=0.001, momentum=0.9)
        '''
        if my_starting_weights and transfer_learning != "none":
            print(f"Loading the weights from {my_starting_weights} ...")
            model_t.load_state_dict(torch.load(my_starting_weights))
            print("Successfully loaded the model checkpoint!")
        '''
        # Sanity check: print layers that are being fine-tuned (not frozen)
        print(check(model_t))

        running_loss = .0
        best_acc = .0
        for epoch in range(epochs):
            print(f"Starting epoch {epoch + 1}")
            for idx, data in tqdm(enumerate(train_loader), total=len(train_loader)):

                # Get the inputs (data is a list of [inputs, labels])
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model_t(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                loss = loss.detach().cpu().numpy()
                inputs = inputs.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                running_loss += loss

            # Evaluate the accuracy after each epoch
            acc = model_t.evaluate(model_t, val_loader, classes, device)
            if acc > best_acc:
                print(f"Better validation accuracy achieved: {acc * 100:.2f}%")
                best_acc = acc
                print(f"Saving this model as: {my_best_model}")
                torch.save(model_t.state_dict(), my_best_model)

    # And here we evaluate the trained model with the test data
    elif mode == "eval":
        print(f"Loading checkpoint from {my_best_model}")
        model_t.load_state_dict(torch.load(my_best_model))
        acc = model_t.evaluate(model_t, test_loader, classes, device)
        print(f"Accuracy on the test (unknown) data: {acc * 100:.2f}%")

    else:
        print("'mode' argument should either be 'train' or 'eval'")

Currently using device:  cuda:0
Loading checkpoint from /content/drive/MyDrive/Semester_Project/best_model.pth


/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Accuracy on the test (unknown) data: 18.94%
